In [1]:
!pip install tensorflow
!pip install git+https://github.com/carlosluis/stable-baselines3@fix_tests 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 839.4 kB/s eta 0:00:0000:0100:07
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 3.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 1.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 1.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 1.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [26]:
import os
import numpy as np
import random
import statistics as stat

import gym
from gym import Env
from gym.spaces import Discrete, Box, MultiDiscrete

from stable_baselines3 import PPO
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [27]:
ENV_NAME = 'TRIA-RL-MODEL3D'

In [73]:
TINI= 60.0; HINI= 30.0; AINI= 100 

TMIN =-40.0; TMAX=120; HMIN=0.0; HMAX=100.0; AMIN=0.0; AMAX=2000.0

EQUI_LEN= 100

R1= -1; R2= -2; R3= 3; NR3= -3

CONST_WAIGHT_VEC = [1,1,1,1]

D = {
     0 : [65.0, 80.0, 50.0, 85.0, 40.0, 90.0], 
     1 : [30.0, 50.0, 20.0, 60.0, 10.0, 70.0], 
     2 : [0.0, 19.0, 200.0, 599.0, 600.0, 2000.0]
    }

D1 = { 0: [65.0, 80.0], 1: [30.0, 50.0], 2: [0.0, 19.0]}
     

In [74]:
class TriaEnv(Env):
    
    def __init__(self):
        self.action_space = MultiDiscrete(np.array([2,2,2,2,2]))
        
        self.observation_space = Box(low=np.array([TMIN, HMIN, AMIN]), high=np.array([TMAX, HMAX, AMAX]))
        
        self.state = [TINI + random.uniform(-2.0, 2.0), HINI + random.uniform(-2.0, 2.0), AINI + random.uniform(-2.0,2.0)]
        
        #print('^^^', self.state, self.action_space)
        
        self.equilibrium_length = EQUI_LEN
        
    def step(self, action):
        
        ap = [1 if e == 1 else -1 for e in action]
        
        actionPrime = [a * b for a,b in zip(ap, CONST_WAIGHT_VEC)]
        
        actionAlgo = [actionPrime[a] - actionPrime[len(actionPrime) -a -1] for a in range(len(actionPrime) // 2)]
        
        actionAlgo.append(actionPrime[len(actionPrime) // 2])                                                              
        
        #print('***',actionAlgo, self.state)
        
        self.state = [a + b for a, b in zip(actionAlgo, self.state)]
        
        #print('&&&', actionAlgo, self.state)
        
        #reduce tria simulation length by 1 second
        self.equilibrium_length -= 1
        
        #reward = [R3 if e >= D[i][0] and e<= D[i][1] else R2 if e >= D[i][2] and e<= D[i][3] else R1 if e >= D[i][4] and e <= D[i][5] else NR3 for i,e in enumerate(self.state)]
        reward = [R3 if e >= D1[i][0] and e <= D1[i][1] else NR3  for i, e in enumerate(self.state)]
        reward = sum(reward)
        #reward = stat.mode(reward)
        #print('$$$', reward)
            
        if self.equilibrium_length <= 0:
            terminated = True
        else:
            terminated = False
            
        info = {}
        print('reward:{} state:{}'.format(reward, self.state))
        return self.state, reward, terminated, False, {}
    
    def render(self):
        pass
    
    def reset(self):
        
        self.state =[TINI + random.uniform(-2.0,2.0),HINI + random.uniform(-2.0,2.0),AINI + random.uniform(-2.0,2.0)]
        #print('@@@', self.state)
        self.equilibrium_length = EQUI_LEN
        
        return self.state, {}
        

In [75]:
env= TriaEnv()

In [76]:
env.observation_space.sample()

array([ 39.216434,  49.487663, 817.18164 ], dtype=float32)

In [77]:
env.action_space.sample()

array([0, 1, 1, 0, 0])

In [78]:
env.state

[61.219723752506034, 30.59976123575708, 98.43616285981304]

In [79]:
#env.reset()

In [80]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    #print(state)
    terminated = False
    score = 0 #[0,0,0] 
    
    while not terminated:
        #env.render()
        action = env.action_space.sample()
        #print(action, terminated , reward)
        #print(env.step(action))
        next_state, reward, terminated, truncated, info = env.step(action) 
        score += reward #[a + b for a, b in zip(reward, score)]
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

reward:-9 state:[58.38864364728077, 26.98755656644505, 102.08851527380187]
reward:-9 state:[58.38864364728077, 28.98755656644505, 101.08851527380187]
reward:-9 state:[58.38864364728077, 28.98755656644505, 100.08851527380187]
reward:-9 state:[56.38864364728077, 28.98755656644505, 99.08851527380187]
reward:-9 state:[56.38864364728077, 26.98755656644505, 100.08851527380187]
reward:-9 state:[54.38864364728077, 24.98755656644505, 101.08851527380187]
reward:-9 state:[54.38864364728077, 26.98755656644505, 100.08851527380187]
reward:-9 state:[54.38864364728077, 24.98755656644505, 101.08851527380187]
reward:-9 state:[56.38864364728077, 24.98755656644505, 100.08851527380187]
reward:-9 state:[56.38864364728077, 24.98755656644505, 101.08851527380187]
reward:-9 state:[54.38864364728077, 24.98755656644505, 100.08851527380187]
reward:-9 state:[56.38864364728077, 22.98755656644505, 101.08851527380187]
reward:-9 state:[54.38864364728077, 20.98755656644505, 102.08851527380187]
reward:-9 state:[54.388643

In [81]:
log_path = os.path.join('training','logs')

In [82]:
#from stable_baselines3 import A2C
#from stable_baselines3.common.vec_env import SubprocVecEnv
#env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1) #, tensorboard_log=log_path)
#model = A2C("MlpPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [83]:
model.learn(total_timesteps=4000)

reward:-3 state:[59.309270578026414, 31.830635855298965, 97.65956255410694]
reward:-3 state:[59.309270578026414, 31.830635855298965, 96.65956255410694]
reward:-3 state:[59.309270578026414, 31.830635855298965, 97.65956255410694]
reward:-9 state:[59.309270578026414, 29.830635855298965, 98.65956255410694]
reward:-9 state:[59.309270578026414, 29.830635855298965, 97.65956255410694]
reward:-9 state:[57.309270578026414, 29.830635855298965, 98.65956255410694]
reward:-9 state:[59.309270578026414, 27.830635855298965, 99.65956255410694]
reward:-9 state:[59.309270578026414, 25.830635855298965, 100.65956255410694]
reward:-9 state:[61.309270578026414, 27.830635855298965, 99.65956255410694]
reward:-9 state:[59.309270578026414, 27.830635855298965, 100.65956255410694]
reward:-9 state:[59.309270578026414, 29.830635855298965, 99.65956255410694]
reward:-9 state:[59.309270578026414, 27.830635855298965, 100.65956255410694]
reward:-9 state:[59.309270578026414, 27.830635855298965, 99.65956255410694]
reward:-9